In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')


In [2]:
spark = (
    SparkSession
    .builder
    .appName("landing_to_bronze")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b047778f-3dcd-4368-a1b6-ac49cf02ac95;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.1.1/spark-avro_2.12-3.1.1.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-avro_2.12;3.1.1!s

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [5]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType, StructType
from dateutil import parser
from delta.tables import *


In [6]:
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/linx_db/dbo/produtos/'
silver_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/dbo/clientes_abordaveis_crm/'

In [17]:
df = (
    spark
    .read
    .format("delta")
    .load(silver_path)
)

In [19]:
clientes_abordaveis = DeltaTable.forPath(spark, silver_path)

In [20]:
sql = '''
    SELECT "Grupo Controle" AS DATA_SOURCE,
    CPF_HASH AS ID_CUSTOMER,
    FLG_GC,
    TRIM(MARCA) as MARCA
    FROM view_clientes_abordaveis
'''

In [22]:
clientes_abordaveis.toDF().createOrReplaceTempView("view_clientes_abordaveis")

In [23]:
result_sql = spark.sql(sql)

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

In [28]:
her_store = result_sql.where(result_sql.MARCA == 'HERING STORE').toPandas()
her_outlet = result_sql.where(result_sql.MARCA == 'HERING OUTLET').toPandas()
dzarm = result_sql.where(result_sql.MARCA == 'DZARM').toPandas()

In [32]:
'her_store' + str(her_store.count())

'her_storeDATA_SOURCE    80403\nID_CUSTOMER    80403\nFLG_GC         80403\nMARCA          80403\ndtype: int64'

In [29]:
her_store.count()

DATA_SOURCE    80403
ID_CUSTOMER    80403
FLG_GC         80403
MARCA          80403
dtype: int64

In [30]:
her_outlet.count()

DATA_SOURCE    20994
ID_CUSTOMER    20994
FLG_GC         20994
MARCA          20994
dtype: int64

In [31]:
dzarm.count()

DATA_SOURCE    8143
ID_CUSTOMER    8143
FLG_GC         8143
MARCA          8143
dtype: int64

In [25]:
teste = result_sql.toPandas()

In [27]:
teste.groupby(['MARCA'])['MARCA'].count()

MARCA
DZARM             8143
HERING OUTLET    20994
HERING STORE     80403
Name: MARCA, dtype: int64

In [8]:
df_pd = df.toPandas()

In [13]:
df_pd.groupby(['MARCA'])['MARCA'].count()

MARCA
DZARM                              6203
DZARM                              1940
HERING OUTLET                      5126
HERING OUTLET                     15868
HERING STORE                      80403
Name: MARCA, dtype: int64

In [12]:
6203 + 1940 + 5126 + 15868 + 80403

109540

In [16]:
def genereteSymlinkManifest(spark:SparkSession, bucket:str) -> None:
    """
    Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
    """
    delta_table = DeltaTable.forPath(spark, bucket)
    delta_table.generate("symlink_format_manifest")

In [17]:
def pathExists(path, spark:SparkSession):
    sc = spark.sparkContext
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
        sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
        sc._jsc.hadoopConfiguration(),
    )
    return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))

In [27]:
def lastDate(delta_path:str, spark:SparkSession):
    if not pathExists(delta_path + 'job_control/last_date', spark):
        try:
            df_temp = (
                spark
                .read
                .format("delta")
                .load(delta_path)
            )

            list_result = (
                df_temp
                .agg({"timestamp_kafka": "max"}).collect()
            )

            last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        except:
            last_date = ('01-01-2021 12:00AM')

        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
    else:
        df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
        last_date = df.collect()[0].asDict()['last_date']
    
    return parser.parse(last_date)

In [30]:
def updateLastDate(df:DataFrame, spark:SparkSession):
    list_result = (
        df
        .agg({"timestamp_kafka": "max"}).collect()
    )

    last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
    df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
    df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
    
    return last_date

In [28]:
# Reading last silver date
last_date = lastDate(silver_path, spark)

In [29]:
print(last_date)

2022-03-17 20:52:26.262000


In [24]:
df_table = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp_kafka")>last_date)
)

In [25]:
df_table.count()

0

In [31]:
# Snapshot
if df_table.where((f.col('__op') == 'r')|(f.col('__op')=='c')).count() > 0:
    df_table_r = df_table.where(f.col('__op') == 'r')
    (
        df_table_r
        .write
        .format("delta")
        .mode("append")
        .option("checkpointLocation", silver_path + "checkpoint")
        .save(silver_path)
    )
    
# Insert and Update
if df_table.where(f.col('__op') != 'r').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    df_table_u = df_table.where(f.col('__op') != 'r')
    df_table_u_unique = df_table_u.orderBy('offset', ascending = False).dropDuplicates(["PRODUTO"])
    (
        silver_table.alias('silver')
        .merge(df_table_u_unique.alias('updates'),'silver.PRODUTO = updates.PRODUTO')
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
    
# Delete
if df_table.where(f.col('__op') == 'd').count() > 0:
    silver_table = DeltaTable.forPath(spark, silver_path)
    silver_table.delete(col('__op') == 'd')  
    

In [33]:
if df_table.count() > 0:
    updateLastDate(df_table, spark)
    genereteSymlinkManifest(spark, silver_path)
    print('maior que 0')

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [25]:
    df_silver_teste = (
        spark
        .read
        .format("delta")
        .load(silver_path)
    )

In [26]:
df_pd = df_silver_teste.toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


In [27]:
df_pd

offset         timestamp_kafka  year  month  day  hour       PRODUTO  \
0      -24161 2022-03-09 13:45:47.668  2022      3    9    13  012N           
1       -7625 2022-03-09 13:45:04.422  2022      3    9    13  014V           
2       -7721 2022-03-09 13:45:04.425  2022      3    9    13  015W           
3      -36316 2022-03-09 13:46:14.155  2022      3    9    13  01YP           
4      -24283 2022-03-09 13:45:47.681  2022      3    9    13  0295           
...       ...                     ...   ...    ...  ...   ...           ...   
97900   -7864 2022-03-09 13:45:06.460  2022      3    9    13  ZMBD           
97901  -42646 2022-03-09 13:46:33.061  2022      3    9    13  ZU6N           
97902  -48612 2022-03-09 13:47:00.239  2022      3    9    13  ZU99           
97903   -9941 2022-03-09 13:45:13.131  2022      3    9    13  ZUJA           
97904  -39683 2022-03-09 13:46:26.686  2022      3    9    13  ZUL9           

      CODIGO_PRECO MATERIAL PERIODO_PCP  ... HR_B2C_CATEGORIA  ID_CEST_NCM  \
0                      None        None  ...             None          NaN   
1                      None        None  ...             None          NaN   
2                      None        None  ...             None          NaN   
3                      None        None  ...             None          NaN   
4                      None        None  ...             None          NaN   
...            ...      ...         ...  ...              ...          ...   
97900                  None        None  ...             None          NaN   
97901                  None        None  ...             None          NaN   
97902                  None        None  ...             None          NaN   
97903                  None        None  ...             None          NaN   
97904                  None        None  ...             None          NaN   

      HR_INDICA_BRINDE HR_PARCEIRO HR_ESTRATEGIA INFO_MODELO __op   __table  \
0                 None        None          None        None    r  PRODUTOS   
1                 None        None          None        None    r  PRODUTOS   
2                 None        None          None        None    r  PRODUTOS   
3                 None        None          None        None    r  PRODUTOS   
4                 None        None          None        None    r  PRODUTOS   
...                ...         ...           ...         ...  ...       ...   
97900             None        None          None        None    r  PRODUTOS   
97901             None        None          None        None    r  PRODUTOS   
97902             None        None          None        None    r  PRODUTOS   
97903             None        None          None        None    r  PRODUTOS   
97904             None        None          None        None    r  PRODUTOS   

      __source_ts_ms __deleted  
0      1646833546855     false  
1      1646833504018     false  
2      1646833504065     false  
3      1646833573323     false  
4      1646833547233     false  
...              ...       ...  
97900  1646833504834     false  
97901  1646833592148     false  
97902  1646833618534     false  
97903  1646833512635     false  
97904  1646833584853     false  

[97905 rows x 165 columns]

In [30]:
df_pd[df_pd["__op"] != "r"]

offset         timestamp_kafka  year  month  day  hour       PRODUTO  \
34      52663 2022-03-16 04:41:02.566  2022      3   16     4  06AK           
130     27191 2022-03-11 02:46:38.729  2022      3   11     2  4A8Y           
147     30130 2022-03-11 04:40:54.406  2022      3   11     4  4E25           
160     50758 2022-03-15 17:50:28.414  2022      3   15    17  4FPQ           
174     78439 2022-03-17 05:35:15.868  2022      3   17     5  54NA           
...       ...                     ...   ...    ...  ...   ...           ...   
97735   28635 2022-03-11 04:40:54.389  2022      3   11     4  H98D           
97736      59 2022-03-08 15:27:34.147  2022      3    8    15  H9CQ           
97757   77726 2022-03-17 05:44:28.417  2022      3   17     5  KFKN           
97794   64643 2022-03-16 17:50:57.216  2022      3   16    17  M06H           
97831   28760 2022-03-11 04:40:54.404  2022      3   11     4  UBL0           

      CODIGO_PRECO MATERIAL PERIODO_PCP  ... HR_B2C_CATEGORIA  ID_CEST_NCM  \
34                     None        None  ...             None          NaN   
130                    None        None  ...             None          NaN   
147                    None        None  ...             None          NaN   
160                    None        None  ...             None          NaN   
174                    None        None  ...         INFANTIL          NaN   
...            ...      ...         ...  ...              ...          ...   
97735                  None        None  ...             None          NaN   
97736                  None        None  ...             None          NaN   
97757                  None        None  ...             None          NaN   
97794                  None        None  ...             None          NaN   
97831                  None        None  ...             None          NaN   

      HR_INDICA_BRINDE HR_PARCEIRO HR_ESTRATEGIA INFO_MODELO __op   __table  \
34                None        None          None        None    u  PRODUTOS   
130               None        None          None        None    u  PRODUTOS   
147               None        None          None        None    u  PRODUTOS   
160               None        None          None        None    u  PRODUTOS   
174               None        None          None        None    u  PRODUTOS   
...                ...         ...           ...         ...  ...       ...   
97735             None        None          None        None    u  PRODUTOS   
97736             None        None          None        None    u  PRODUTOS   
97757             None        None          None        None    u  PRODUTOS   
97794             None        None          None        None    u  PRODUTOS   
97831             None        None          None        None    u  PRODUTOS   

      __source_ts_ms __deleted  
34     1647405657487     false  
130    1646966792393     false  
147    1646973646740     false  
160    1647366624380     false  
174    1647495310903     false  
...              ...       ...  
97735  1646973646740     false  
97736  1646753247243     false  
97757  1647495865523     false  
97794  1647453051560     false  
97831  1646973646740     false  

[1975 rows x 165 columns]